##Exploratory Data Analysis

####Phase 1

    validate data/ normalize
    do plots first 
    univariate discriptive stats
    univariate plots 
    bivariate discriptive stats (cross tab stable)
    bivariate plots 


####Phase 2
    GOAL: we want to moves users from their current segment to the next segment up the number line
    segment users between anomoly spikes 
    1 - 6 purchase should all be seperate groups 
    7 - 12 purcahse should all be one group
    12 should be its own group because it's an anomoly 
    13 - 24 purchases should all be one group 
    25 should its own group because it's also an anomoly
    26 - 100 is it's own group
    100 + is its own group 
    power users are the anomolous groups

    find out why bin 6 and 12 are disproportionally large 

####Phase 3
    historical cohort analysis 
    all user who have bought one bottle of wine
    keep track of users who churn and who buy a second bottle of wine
    how is that first bottle of wine related to that second bottle of wine?
    we want to generalize this to understand how to go from k to k + 1 bottles

##Notes - Meeting with Brain Nov 4, 1:00 pm

sort all graphs

Read up on: 

LDA for wines --> topic modeling --> micro genres 

For later, look into twitter time series anomoly detection (after proto-rec sys is built)

###Phase 1

In [1]:
reset -f -s

In [2]:
# print_function turns print into a python 3 function 
# this enables python to print funky text i.e. f\xn --> fun
from __future__ import (division,
                        absolute_import,
                        print_function,
                        unicode_literals
                        )
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import ftfy
import math
import operator
from collections import defaultdict, Counter
import seaborn as sns
%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning:

axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.



In [3]:
def clean_text(text_array):
        # review what is going on here with Brian -- make sure you understand why this works!
    clean_text = []
    for i, token in enumerate(text_array):
        try:
            clean_text.append(ftfy.fix_encoding(ftfy.fix_text(unicode(token,"utf-8"))))
        except TypeError:
            print(token)
            print(i)
            break
        except UnicodeDecodeError:
            try:
                guessed_encoding = ftfy.guess_bytes(token)[1]
                clean_text.append(ftfy.fix_text(unicode(token,guessed_encoding)))
                continue
            except:
                print(token)
                print(i)
                break
    return clean_text

In [4]:
def sort_dict_to_list(data_dict, sort_by):
    '''Input: dictionary
       Input: sort_by, if 1, then sort by value, if 2 sort by key
       Output: returns a sorted key and value list'''
    # sort dictionary by value --> .itemgetter(1)
    sorted_data = sorted(data_dict.items(), key=operator.itemgetter(sort_by), reverse=True)

    # move sorted dates and sorted date counts to lists
    k_datum = []
    v_count_datum = []
    for datum in sorted_data:
        k_datum.append(datum[0])
        v_count_datum.append(datum[1])
    return k_datum, v_count_datum

In [5]:
path1 = "/Users/Alexander/cap_stone_project_wine/data/Customers.txt"
customers = pd.read_csv(path1, sep = "\t")

In [6]:
path2 = "/Users/Alexander/cap_stone_project_wine/data/Ratings.txt"
ratings = pd.read_csv(path2, sep = "\t")

In [7]:
path3 = "/Users/Alexander/cap_stone_project_wine/data/Purchases.txt"
purchases = pd.read_csv(path3, sep = "\t")

In [8]:
path4 = "/Users/Alexander/cap_stone_project_wine/data/Products.txt"
products = pd.read_csv(path4, sep = "\t")

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2871: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



##Products

In [9]:
products.head(10)

,ProductKey,ProductId,ProductType,CountryOfOrigin,Region,Appellation,Varietal,Class,WineStyle,Vineyard,Vintage,ProductName,ProductNameWithVintage,HighestRating,Price,WarehouseState
0,2,127554,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2010,Turnbull Cabernet Sauvignon,Turnbull Cabernet Sauvignon 2010,90,42.99,NY
1,3,133070,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2012,Turnbull Cabernet Sauvignon,Turnbull Cabernet Sauvignon 2012,91,52.99,NY
2,4,114037,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2009,Turnbull Cabernet Sauvignon,Turnbull Cabernet Sauvignon 2009,88,44.99,NY
3,5,116887,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Sauvignon Blanc,White Wines,White Styles: Fruity & Smooth,Turnbull,2011,Turnbull Sauvignon Blanc,Turnbull Sauvignon Blanc 2011,0,24.99,NY
4,6,145898,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Sauvignon Blanc,White Wines,White Styles: Light & Crisp,Turnbull,2014,Turnbull Sauvignon Blanc,Turnbull Sauvignon Blanc 2014,0,24.99,NY
5,7,109508,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Other Red Blends,Red Wines,Red Styles: Big & Bold,Turnbull,2009,Turnbull Old Bull Red Blend,Turnbull Old Bull Red Blend 2009,0,19.97,NY
6,8,110277,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Merlot,Red Wines,Red Styles: Smooth & Supple,Turnbull,2007,Turnbull Merlot,Turnbull Merlot 2007,0,35.99,NY
7,9,113520,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2008,Turnbull Black Label Cabernet Sauvignon,Turnbull Black Label Cabernet Sauvignon 2008,0,99.00,NY
8,10,134691,Wine,NEW ZEALAND,New Zealand,Martinborough,Pinot Gris/Grigio,White Wines,White Styles: Light & Crisp,Tablelands,2013,Tablelands Pinot Gris,Tablelands Pinot Gris 2013,90,16.99,NY
9,11,133601,Wine,NEW ZEALAND,New Zealand,Martinborough,Sauvignon Blanc,White Wines,White Styles: Light & Crisp,Tablelands,2013,Tablelands Sauvignon Blanc,Tablelands Sauvignon Blanc 2013,90,15.99,NY


In [10]:
products.ProductName.unique()

array(['Turnbull Cabernet Sauvignon', 'Turnbull Sauvignon Blanc',
       'Turnbull Old Bull Red Blend', ...,
       'Reginato Torrontes / Chardonnay Brut NV',
       'Waterstone Carneros Pinot Noir',
       'Amala Springs Sonoma County Zinfandel'], dtype=object)

###ProductId

In [11]:
# total number of wines
products.ProductId.count()

394026

In [12]:
# numner of unique wines
len(products.ProductId.unique())

110511

###ProductType

In [13]:
products.ProductType.count()

394026

In [14]:
products.ProductType.unique()

array(['Wine'], dtype=object)

###WarehouseState

In [15]:
print(products.WarehouseState.unique())
print("{} warehouse states -- not all in the USA".format(len(products.WarehouseState.unique())))

['NY' 'MA' 'NC' 'NJ' 'MI' 'CA' 'VA' 'CT' 'MT' 'IN' 'OH' 'FL' 'OR' 'TX' 'JA'
 'WA' 'AZ' 'WI' 'RI' 'IA' 'VT' 'CO' 'ME' 'IL' 'HI' 'WG' 'Fl']
27 warehouse states -- not all in the USA


In [16]:
state_count = Counter(products.WarehouseState.values)

In [17]:
sort_by_row = 1
states, count_states = sort_dict_to_list(state_count,sort_by_row)

In [18]:
data = [
    go.Bar(
        x=states,
        y=count_states,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by WarehouseState',
    
        yaxis=dict(
        title='Frequency of wine'
    ),
        xaxis=dict(
        title="State"
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_WarehouseState = py.iplot(fig, filename='style-bar')

In [19]:
plot_url_WarehouseState

###WineStyles

In [20]:
products.WineStyle.unique()

array(['Red Styles: Big & Bold', 'White Styles: Fruity & Smooth',
       'White Styles: Light & Crisp', 'Red Styles: Smooth & Supple',
       'Red Styles: Earthy & Spicy', 'White Styles: Rich & Creamy',
       'Red Styles: Light & Fruity', nan], dtype=object)

In [21]:
print (np.sum(products.WineStyle.isnull() == True))
print (np.sum(products.WineStyle.isnull() == False))
print ("\n{:.4} Wine Styles are nan".format(np.sum(products.WineStyle.isnull() == False)/np.sum(products.WineStyle.isnull() == True)))

258299
135727

0.5255 Wine Styles are nan


In [22]:
style_count = Counter(products.WineStyle.values)

In [23]:
sort_by_row = 1
style, count_style = sort_dict_to_list(style_count,sort_by_row)

In [24]:
data = [
    go.Bar(
        x=style,
        y=count_style,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Count of Wine by Style',
    
        yaxis=dict(
        title='Count of wine'
    ),
        xaxis=dict(
        title="Style"
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_style = py.iplot(fig, filename='style-bar')

In [25]:
# null vs not null
# all non-null values 
# fork data and find out why nulls are so numerous
# I need to impute the null values -- very  important !!!!
# look at other wines with similar/identical attributes and infer correct style
plot_url_style

#To Do

###Imput Null wine styles
    Will have to cluster similar wines to find wine style label for nulls

In [26]:
# region, appellation, varietal, class, price, highestratings - subgenre, some combination of these features
# vineyard, vintage may not be predictive
# style will soon be dropped 
# stay away from topic modeling, use the above features for creating sub-genres 

products.head(10)

,ProductKey,ProductId,ProductType,CountryOfOrigin,Region,Appellation,Varietal,Class,WineStyle,Vineyard,Vintage,ProductName,ProductNameWithVintage,HighestRating,Price,WarehouseState
0,2,127554,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2010,Turnbull Cabernet Sauvignon,Turnbull Cabernet Sauvignon 2010,90,42.99,NY
1,3,133070,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2012,Turnbull Cabernet Sauvignon,Turnbull Cabernet Sauvignon 2012,91,52.99,NY
2,4,114037,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2009,Turnbull Cabernet Sauvignon,Turnbull Cabernet Sauvignon 2009,88,44.99,NY
3,5,116887,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Sauvignon Blanc,White Wines,White Styles: Fruity & Smooth,Turnbull,2011,Turnbull Sauvignon Blanc,Turnbull Sauvignon Blanc 2011,0,24.99,NY
4,6,145898,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Sauvignon Blanc,White Wines,White Styles: Light & Crisp,Turnbull,2014,Turnbull Sauvignon Blanc,Turnbull Sauvignon Blanc 2014,0,24.99,NY
5,7,109508,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Other Red Blends,Red Wines,Red Styles: Big & Bold,Turnbull,2009,Turnbull Old Bull Red Blend,Turnbull Old Bull Red Blend 2009,0,19.97,NY
6,8,110277,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Merlot,Red Wines,Red Styles: Smooth & Supple,Turnbull,2007,Turnbull Merlot,Turnbull Merlot 2007,0,35.99,NY
7,9,113520,Wine,UNITED STATES OF AMERICA,California,Napa Valley,Cabernet Sauvignon,Red Wines,Red Styles: Big & Bold,Turnbull,2008,Turnbull Black Label Cabernet Sauvignon,Turnbull Black Label Cabernet Sauvignon 2008,0,99.00,NY
8,10,134691,Wine,NEW ZEALAND,New Zealand,Martinborough,Pinot Gris/Grigio,White Wines,White Styles: Light & Crisp,Tablelands,2013,Tablelands Pinot Gris,Tablelands Pinot Gris 2013,90,16.99,NY
9,11,133601,Wine,NEW ZEALAND,New Zealand,Martinborough,Sauvignon Blanc,White Wines,White Styles: Light & Crisp,Tablelands,2013,Tablelands Sauvignon Blanc,Tablelands Sauvignon Blanc 2013,90,15.99,NY


In [27]:
import sklearn

In [28]:
# Check that Class has no nulls
print (Counter(products.Class.notnull()))
print (products.Class.count())

Counter({True: 394026})
394026


In [29]:
# Check that Varietal has no nulls
print (Counter(products.Varietal.notnull()))
print (products.Varietal.count())

Counter({True: 394026})
394026


In [30]:
# Check that Vineyard has no nulls
print (Counter(products.Vineyard.notnull()))
print (products.Vineyard.count())

Counter({True: 394026})
394026


###Vineyard

In [31]:
print(products.Vineyard.unique())
print("\n{} total number of vineyards".format(len(products.Vineyard.unique())))

['Turnbull' 'Tablelands' 'Evodia' ..., "Veterans' Spirit" 'Calvano'
 'Amala Springs']

8352 total number of vineyards


In [32]:
clean_vineyards = clean_text(products.Vineyard.values)

In [33]:
vineyards_count = Counter(clean_vineyards)

In [34]:
sort_by_row = 1
vineyards, count_vineyards = sort_dict_to_list(vineyards_count,sort_by_row)

In [35]:
data = [
    go.Bar(
        x=vineyards,
        y=count_vineyards,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Vineyard',
    
        yaxis=dict(
        title='Frequency of wine'
        )
)
fig = go.Figure(data=data, layout=layout)
plot_url_vineyard = py.iplot(fig, filename='style-bar')

In [36]:
plot_url_vineyard

###CountryOfOrigin

In [37]:
# string look clean
products.CountryOfOrigin.unique()

array(['UNITED STATES OF AMERICA', 'NEW ZEALAND', 'SPAIN', 'FRANCE',
       'GERMANY', 'AUSTRALIA', 'ITALY', 'CHILE', 'ARGENTINA',
       'SOUTH AFRICA', 'GREECE', 'AUSTRIA', 'PORTUGAL', 'SLOVENIA',
       'Unknown', 'ISRAEL', 'CANADA', 'JAPAN', 'HUNGARY', 'SWITZERLAND'], dtype=object)

In [38]:
origin_count = Counter(products.CountryOfOrigin.values)

In [39]:
sort_by_row = 1
origin, count_origin = sort_dict_to_list(origin_count,sort_by_row)

In [87]:
data = [
    go.Bar(
        x=origin,
        y=count_origin,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Country of Origin',
    
        yaxis=dict(
        title='Frequency of wine'
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_origin_origin = py.iplot(fig, filename='style-bar')

In [88]:
plot_url_origin_origin

###Region

In [42]:
# Assuming meaningful distinction bteween JA and non-JA locations --> strings look clean
# what's the difference between 'New Zealand' and 'New Zealand(JA)'
products.Region.unique()

array(['California', 'New Zealand', 'Spain', 'France - Other regions',
       'Oregon', 'Germany', 'Other US', 'Australia', 'Italy',
       'South America', 'France - Bordeaux', 'France - Rhone',
       'Washington', 'South Africa', 'Greece', 'Other Europe', 'Portugal',
       'Various Regional', 'Israel', 'Canada', 'Japan', 'Unknown',
       'Californian(JA)', 'Italian(JA)', 'Spanish(JA)', 'French(JA)',
       'Mexico', 'Various Regional(JA)', 'Washington(JA)', 'German(JA)',
       'Oregon(JA)', 'South American(JA)', 'Australian(JA)',
       'New Zealand(JA)'], dtype=object)

In [43]:
region_count = Counter(products.Region.values)

In [44]:
sort_by_row = 1
region, count_region = sort_dict_to_list(region_count,sort_by_row)

In [90]:
data = [
    go.Bar(
        x=region,
        y=count_region,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Region',
    
        yaxis=dict(
        title='Frequency of wine'
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_region = py.iplot(fig, filename='style-bar')

In [91]:
plot_url_region

###Appellation

In [97]:
# strings look clean
# what's the difference between 'Other New Zealand' and 'Other New Zealand(JA)'
products.Appellation.unique()[0:20]

array(['Napa Valley', 'Martinborough', 'Other Spain',
       'Languedoc-Roussillon', 'Rias Baixas', 'Other Oregon',
       'Other California', 'Mosel-Saar-Ruwer', 'New York', 'Sonoma County',
       'Yarra Valley', 'Russian River', 'Carneros', 'Rioja',
       'Other Australia', 'Tuscany', 'Veneto', 'Central Coast', 'Chile',
       'Other Bordeaux'], dtype=object)

In [98]:
clean_appell = clean_text(products.Appellation.values)

In [99]:
appellation_count = Counter(clean_appell)

In [100]:
sort_by_row = 1
appell, count_appell = sort_dict_to_list(appellation_count, sort_by_row)

In [101]:
data = [
    go.Bar(
        x=appell,
        y=count_appell,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by appellation',
    
        yaxis=dict(
        title='Frequency of wine'
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_appellation = py.iplot(fig, filename='style-bar')

In [102]:
plot_url_appellation

###Varietal

In [53]:
# string look clean
products.Varietal.unique()[0:20]

array(['Cabernet Sauvignon', 'Sauvignon Blanc', 'Other Red Blends',
       'Merlot', 'Pinot Gris/Grigio', 'Grenache', 'Syrah/Shiraz',
       'Rhone Red Blends', 'Other White Wine', 'Cabernet Franc',
       'Albarino', 'Pinot Noir', 'Zinfandel', 'Gewurztraminer', 'Riesling',
       'Chardonnay', 'Tempranillo', 'Sangiovese', 'Other Red Wine',
       'Bordeaux Red Blends'], dtype=object)

In [54]:
clean_varietal = clean_text(products.Varietal.values)

In [55]:
varietal_count = Counter(clean_varietal)

In [105]:
sort_by_row = 1
var, count_var = sort_dict_to_list(varietal_count, sort_by_row)

In [106]:
data = [
    go.Bar(
        x=var,
        y=count_var,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Varietal',
    
        yaxis=dict(
        title='Frequency of wine'
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_varietal = py.iplot(fig, filename='style-bar')

In [107]:
plot_url_varietal

###Vintage

In [59]:
# Ask Cam if he knows about the anomalies "4", "1", "3", "2201"...
# Will likely have to replace with best guess based on similarity score 
products.Vintage.unique()

array([2010, 2012, 2009, 2011, 2014, 2007, 2008, 2013, 2006, 2005, 2004,
       2002, 2003,    0, 1994, 2001, 1995, 1999, 1996, 2000, 1991, 1966,
       1998, 1983, 1997, 1985, 1990, 1988, 1979, 1989, 1982, 1981, 1986,
       1993, 1992, 2015, 1972, 1984, 1987, 1977, 1975, 1969, 1961, 1971,
       1980, 1934, 1978, 1976, 1968, 1845, 1958, 1964, 1954, 1973,  199,
       1922, 1967, 1970,  200, 1963, 1920, 1962,    4, 1910, 1875, 1953,
       1944,   97, 1974, 1950, 1949, 1959, 1945, 1921, 1937, 1955, 1947,
       1933, 1908,   94,    3, 1912, 1960, 1911, 1952,    1, 1940, 1965,
       2201, 1948, 1941, 1943, 1951, 1929])

In [60]:
vintage = products.Vintage.values

In [61]:
vintage.sort()

In [62]:
start_year = 1994
end_year = 2015

In [63]:
vintage = vintage[vintage >= start_year]

In [64]:
vintage = vintage[vintage <= end_year]

In [65]:
vintage_count = Counter(vintage)

In [66]:
data = [
    go.Bar(
        x=vintage_count.keys(),
        y=vintage_count.values(),
        
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Vintage',
    
        yaxis=dict(
        title='Frequency of wine'
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_vintage = py.iplot(fig, filename='style-histogram')

In [67]:
# truncate vintage - want years 1994 and above
# i.e. do not recommend a 60 year old, $40,000  bottle of wine
plot_url_vintage

###Class

In [111]:
products.Class.unique()

array(['Red Wines', 'White Wines', 'Champagne & Sparkling', 'Ros\xe9 Wine',
       'Dessert, Fortified & Fruit Wines', 'Collections', 'Sake',
       'Accessories', 'Rum', 'Liqueur', 'Brandy', 'Vodka', 'Tequila',
       'Whisky', 'Catalogs', 'Glassware'], dtype=object)

In [112]:
clean_class = clean_text(products.Class.values)

In [113]:
count_class = Counter(clean_class)

In [114]:
sort_by_row = 1
Class, count_class = sort_dict_to_list(count_class,sort_by_row)

In [117]:
data = [
    go.Bar(
        x=Class,
        y=count_class,
            marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Class',
    
        yaxis=dict(
        title='Frequency of wine'
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_class = py.iplot(fig, filename='style-bar')

In [118]:
plot_url_class

###Price
    somethings wrong with the plot
    vertical values should in the 1000's not in the 1's and 10's

In [74]:
clean_price = map(int,np.ceil(products.Price.values))

In [75]:
price_count = Counter(clean_price)

In [76]:
sort_by_key = 0
price, count_price = sort_dict_to_list(price_count, sort_by_key)

In [77]:
data = [
    go.Bar(
        x=price,
        y=count_price,
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Price',
    
        yaxis=dict(
        title='Frequency of Wine'
    
    ),
        xaxis=dict(
        title="Price"
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_price = py.iplot(fig, filename='style-bar')

In [78]:
# wine that is priced at round factors of 10 sell very well
# expore this anomaly 
#plot_url_price

###Highest Ratings

In [79]:
products.HighestRating.unique()

array([ 90,  91,  88,   0,  89,  93,  92,  94,  87,  96,  95,  98,  97,
       100,  99,  86,  85,  84,  18])

In [80]:
rating_count = Counter(products.HighestRating.values)

In [81]:
data = [
    go.Bar(
        x=rating_count.keys(),
        y=rating_count.values(),
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Highest Rating',
    
        yaxis=dict(
        title='Frequency of Wine'
    
    ),
        xaxis=dict(
        title="Highest Rating"
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_rating = py.iplot(fig, filename='style-bar')

In [82]:
# should I throw out the zeros? 
# Answer is Yes! Note why I ulimately decide to throw it away. 
#plot_url_rating

In [83]:
temp_rating = dict()
for k, v in rating_count.iteritems():
    if k > 80:
        temp_rating[k] = v

In [84]:
data = [
    go.Bar(
        x=temp_rating.keys(),
        y=temp_rating.values(),
        marker=dict(
        color="#ce2c4b"
    ),
    )
]
layout = go.Layout(
    title='Frequency of Wine by Highest Rating',
    
        yaxis=dict(
        title='Frequency of Wine'
    
    ),
        xaxis=dict(
        title="Highest Rating"
    )
)
fig = go.Figure(data=data, layout=layout)
plot_url_rating_temp = py.iplot(fig, filename='style-bar')

In [85]:
#plot_url_rating_temp